In [36]:
import pandas as pd
import numpy as np 
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [37]:
df = pd.read_csv("C:/Users/chnvd/OneDrive/Desktop/Fake Logo Detection/file_mapping.csv")
df = df[['Filename','Brand Name','Label']]
df

,Filename,Brand Name,Label
0,output\Cowbell\000001.jpg,Cowbell,Genuine
1,genLogoOutput\Rolex\000003.jpg,Rolex,Fake
2,output\PlayStation\000005.jpg,PlayStation,Genuine
3,output\YouTube\000004.jpg,YouTube,Genuine
4,genLogoOutput\Lays\000004.jpg,Lays,Fake
...,...,...,...
820,output\Google\000002.jpg,Google,Genuine
821,output\Nescafe\000004.jpg,Nescafe,Genuine
822,genLogoOutput\Lays\000003.jpg,Lays,Fake
823,genLogoOutput\Gillette\scal_000002.jpg,Gillette,Fake


In [38]:
text_file_path = "C:/Users/chnvd/OneDrive/Desktop/Fake Logo Detection/Logos.txt"
with open(text_file_path,'r') as file:
    brand_names = file.read().splitlines()
brand_names

['Bic',
 'Samsung',
 'Pepsi',
 'Lays',
 'Mars',
 'MnM',
 'Mtn dew',
 'Oreo',
 'Heinz',
 'Marvel',
 'PlayStation',
 'Chevrolet',
 'Burger King',
 'Hp',
 'Fila',
 'Microsoft',
 'Chrome',
 'NASA',
 'Reebok',
 'Oral b',
 'Cowbell',
 'Peak milk',
 'Twitter',
 'Google',
 'Adidas',
 'Android',
 'Nutella',
 'Puma',
 'Pringles',
 'Sprite',
 'Tesla',
 'Netflix',
 'Zara',
 'Huawei',
 'YouTube',
 'Lipton',
 'Nestle',
 'Nescafe',
 'Krisspy Kreme',
 'Mtn',
 'Nestle milo',
 'Guinness',
 'Goya oil',
 'Gillette',
 'Starbucks',
 'Nike',
 'Dominos',
 'McDonald',
 'Coca Cola',
 'Apple',
 'Levis',
 'Fila',
 'Lg',
 'Amazon',
 'Ariel',
 'Cadbury',
 'Mercedes Benz',
 'Rolex',
 'Pepsi',
 'Pay pal',
 'Kfc',
 'BMW',
 'Honda',
 'Instagram',
 'Tiktok',
 'Twitter']

In [39]:
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])
df.head()

,Filename,Brand Name,Label
0,output\Cowbell\000001.jpg,Cowbell,1
1,genLogoOutput\Rolex\000003.jpg,Rolex,0
2,output\PlayStation\000005.jpg,PlayStation,1
3,output\YouTube\000004.jpg,YouTube,1
4,genLogoOutput\Lays\000004.jpg,Lays,0


In [40]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Brand Name'], random_state=42)

In [41]:
train_df.shape # (660,3)
test_df.shape # (165,3)
train_df.head()

,Filename,Brand Name,Label
289,genLogoOutput\Lays\000001.jpg,Lays,0
77,genLogoOutput\Mtn\scal_000002.jpg,Mtn,0
577,genLogoOutput\Nutella\000001.jpg,Nutella,0
590,genLogoOutput\McDonald\scal_000003.jpg,McDonald,0
434,genLogoOutput\Krisspy Kreme\scal_000003.jpg,Krisspy Kreme,0


In [42]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path) # opens the image with specified path
    image = image.resize(target_size) # resizes the image with target_size given 
    image_array = np.array(image) / 255.0  # Normalize pixel values
    return image_array

In [43]:
train_images = np.array([load_and_preprocess_image(img_path) for img_path in train_df['Filename']])
test_images = np.array([load_and_preprocess_image(img_path) for img_path in test_df['Filename']])

In [44]:
test_images.shape

(165, 224, 224, 3)

In [45]:
train_labels = train_df['Label'].values
test_labels = test_df['Label'].values

In [46]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [47]:
model = Sequential()

model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

Epoch 1/10
21/21 [==============================] - 11s 518ms/step - loss: 0.4916 - accuracy: 0.7015 - val_loss: 0.4473 - val_accuracy: 0.7333
Epoch 2/10
21/21 [==============================] - 11s 507ms/step - loss: 0.4580 - accuracy: 0.7515 - val_loss: 0.4214 - val_accuracy: 0.7758
Epoch 3/10
21/21 [==============================] - 11s 511ms/step - loss: 0.4278 - accuracy: 0.7985 - val_loss: 0.4398 - val_accuracy: 0.7636
Epoch 4/10
21/21 [==============================] - 11s 514ms/step - loss: 0.3825 - accuracy: 0.8364 - val_loss: 0.3882 - val_accuracy: 0.8242
Epoch 5/10
21/21 [==============================] - 11s 503ms/step - loss: 0.3283 - accuracy: 0.8606 - val_loss: 0.2987 - val_accuracy: 0.8909
Epoch 6/10
21/21 [==============================] - 10s 501ms/step - loss: 0.2526 - accuracy: 0.8894 - val_loss: 0.5306 - val_accuracy: 0.7273
Epoch 7/10
21/21 [==============================] - 11s 504ms/step - loss: 0.2988 - accuracy: 0.8621 - val_loss: 0.2956 - val_accuracy: 0.8909

In [50]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

6/6 [==============================] - 1s 120ms/step - loss: 0.1868 - accuracy: 0.9333
Test Loss: 0.18676941096782684, Test Accuracy: 0.9333333373069763


In [52]:
#new_images_paths = ['genLogoOutput/Rolex/000003.jpg','output/PlayStation/000005.jpg','output/YouTube/000004.jpg','genLogoOutput/Lays/000004.jpg','output/Tiktok/000004.jpg']
new_image_path = 'C:/Users/chnvd/OneDrive/Desktop/Fake Logo Detection/genLogoOutput/Adidas/f74fe6235856d9f46a56baa02ab47165.jpg'
new_image = np.array([load_and_preprocess_image(new_image_path)])

predictions = model.predict(new_image)

classes_prediction = (predictions >= 0.8).astype(int)
for prediction in classes_prediction:
    if prediction==1:
        print("Genuine")
    else:
        print("Fake")

1/1 [==============================] - 0s 107ms/step
Genuine
